In [62]:
import pandas as pd
import numpy as np
#from parameter_estimator import ParameterEstimator as pe
from robot import RobotDescription
import utils
from collections import deque
import cv2
import random
import math as m
from itertools import combinations
import pickle
from pytransform3d.transform_manager import *
import datetime

In [63]:
# functions to convert from and to spherical coordinates
def asSpherical(x, y, z):
    r = m.sqrt(x*x + y*y + z*z)
    theta = m.acos(z/r)*180 / m.pi  # to degrees
    phi = m.atan2(y, x)*180 / m.pi
    return r, theta, phi


def asCartesian(r, theta, phi):
    theta = theta * m.pi/180  # to radian
    phi = phi * m.pi/180
    x = r * m.sin(theta) * m.cos(phi)
    y = r * m.sin(theta) * m.sin(phi)
    z = r * m.cos(theta)
    return x, y, z

In [64]:
# generate the poses of markers
ids = list(range(30))  # marker ids to use
dict_T_WM = {}

for marker_id in ids:  # generate random poses for virtual markers
    r = np.random.normal(3, 0.12, 1)[0]
    theta = np.random.uniform(0, 90, 1)[0]
    phi   = np.random.uniform(0, 360, 1)[0]
    x, y, z = asCartesian(r, theta, phi)
    dict_T_WM[marker_id]= utils.H_rvec_tvec(np.array([0.0, 0, 0]), np.array([x, y, z]))


In [65]:
# generate robot configurations

# import trajectory
traj_file="/home/armin/catkin_ws/src/kident2/src/traj.csv"
try:
    df = pd.read_csv(traj_file)
except Exception as e:
    print(e)
traj = df.to_numpy()  # shape: (num_joints, num_traj_points)
traj = traj[:, 1:]  # delete header
traj = traj.T
#print(traj)

In [66]:
# import robot values
theta_all = RobotDescription.dhparams["theta_nom"]
d_all = RobotDescription.dhparams["d_nom"]
r_all = RobotDescription.dhparams["r_nom"]
alpha_all = RobotDescription.dhparams["alpha_nom"]


In [67]:
def add_error_to_pose_measurement(pose_matrix, sigma_t, sigma_r):
    rvec, tvec = utils.mat2rvectvec(pose_matrix)
    rvec_n = np.array(rvec) + np.random.normal(0,sigma_r/1000,3)
    tvec_n = np.array(tvec) + np.random.normal(0,sigma_t/1000,3)
    return utils.H_rvec_tvec(rvec_n,tvec_n)

In [73]:
list_obs = []

add_error = True
error_factor_r = 1
error_factor_t = 1

# generate pseudo measurement data
traj_long = np.concatenate([traj])
for q in traj_long:
    
    # choose 4 random markers for this configuration
    markers = random.choices(ids, k=4)
    
    for marker in markers:
        tm = TransformManager()
        
        T_WM = dict_T_WM[marker]
        tm.add_transform("marker", "world", T_WM)
        
        joint_tfs = RobotDescription.get_joint_tfs(q)
        
        for tf in joint_tfs:
            tm.add_transform(tf['from_frame'], tf['to_frame'], tf['mat'])
        
        last_frame = RobotDescription.dhparams['num_cam_extrinsic'] + RobotDescription.dhparams['num_joints']
        
        T_CM = tm.get_transform('marker', str(last_frame))
        
        if add_error:
            T_CM = add_error_to_pose_measurement(T_CM, error_factor_t, error_factor_r)
        obs = {"marker_id": marker,
               "mat": T_CM,
               "t": 0,
               "q": q,
               "interp_dist": 0}
        list_obs.append(obs)
        

df = pd.DataFrame(list_obs)

In [74]:
# save the observations into a pickle file
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
if add_error:
    observations_file_str = 'observations_simulated_' + timestamp + '_errors_rt_' + str(error_factor_r) + '_' + str(error_factor_t) + '.p'
else:
    observations_file_str = 'observations_simulated_' + timestamp + '.p'
print(observations_file_str)
pd.to_pickle(df, observations_file_str)


observations_simulated_20240128_155015_errors_rt_1_1.p
